In [1]:
# importing dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# creating driver and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_mac64.zip
Driver has been saved in cache [/Users/nicksheets/.wdm/drivers/chromedriver/mac64/92.0.4515.107]


In [3]:
# URL of news page to be scraped and visit it with browser
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)

In [12]:
# creating HTML object
news_html = browser.html

# initiating soup object for news scrape
news_soup = BeautifulSoup(news_html, 'html.parser')

# searching html for latest news
latest_news = news_soup.find('section', class_='image_and_description_container')
title = latest_news.find("div", class_='content_title').text
paragraph = latest_news.find("div", class_='article_teaser_body').text
paragraph
# getting title and paragraph for latest article
# title = latest_news.find('div', class_='content_title').text
# paragraph = latest_news.find('div', class_='article_teaser_body').text

"The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. "

In [13]:
# URL of image page to be scraped and visit it with browser
mars_url = 'https://spaceimages-mars.com'
browser.visit(mars_url)

In [20]:
# HTML object
html = browser.html

# instatiating beautiful soup object and parsing using lxml
image_soup = BeautifulSoup(html, 'lxml')

# accessing the image url
space_image_url = image_soup.find('a', class_='showimg fancybox-thumbs')['href']

# attaching the image url to sliced page url
image_url = f"https://spaceimages-mars.com/{space_image_url}"
image_url


'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [ ]:
# URL of facts page to be scraped and visit it with browser
facts_url = 'https://galaxyfacts-mars.com'

# parsing html using pandas
facts_tables = pd.read_html(facts_url)
facts_tables

In [ ]:
# turning pandas parse into a dataframe
facts_df = facts_tables[0]

# cleaning up the table for printing
facts_df = facts_df.rename(columns={0: " ", 1: "Mars"})
facts_df.set_index(" ", inplace=True)

# converting df to html with pandas
facts_html = facts_df.to_html()

In [ ]:
print(facts_html)

In [ ]:
# URL of image page to be scraped and visit it with browser
hemi_image_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_image_url)

In [ ]:
# URL of image page to be scraped and visit it with browser
hemi_image_url = 'https://marshemispheres.com/'
browser.visit(hemi_image_url)


In [ ]:
# HTML object
html = browser.html

# instatiating beautiful soup object and parsing with html
hemi_image_soup = BeautifulSoup(html, 'html.parser')

# accessing the names of the links to click and storing in a list to iterate over
hemi_names = hemi_image_soup.find_all('div', class_='description')
hemi_name_list = []
for name in hemi_names:
    hemi_name_list.append(name.a.h3.text)

# creating list for dicts for return
hemi_list = []

# looping over the pages, scraping, creating dict, and adding to list
for name in hemi_name_list:
        try:
            # clicking into the page to get the image url
            browser.links.find_by_partial_text(name).click()
            
            # getting image_url
            html = browser.html
            image_url_soup = BeautifulSoup(html, 'html.parser')
            dl_image_url = image_url_soup.find_all('dd')[1].a['href']
            image_url = f"{dl_image_url}/full.jpg"
            
            # getting title from name
            title = name[0:-9]
            
            # creating dict and adding to list
            entry = {'title': title, 'image_url': image_url, 'dl_image_url': dl_image_url}
            hemi_list.append(entry)
            
            # redirecting back to the main page to continue loop
            browser.visit(hemi_image_url)
            
            print("Scrape successful!")
            
        except:
            print("Scrape unsuccessful!")